# ML Surrogates for Chemical Processes with OMLT
This notebook illustrates the use of TensorFlow Keras and OMLT to produce an ML surrogate based on data from a chemical process flowsheet.

There are several reasons to build surrogate models for complex processes, even when higher fidelity models already exist (e.g., reduce model size, improve convergence reliability, replace models with externally compiled code and make them fully-equation oriented).

In this example, we have an existing model for an auto-thermal reformer flowsheet that has been built using the IDAES-PSE package. IDAES-PSE is a Python package that is built on Pyomo and provides a framework for equation-oriented modeling and analysis of advanced energy systems. We use this package to generate data for our systems, and then we utilize this data in an optimization problem with OMLT. To learn more about IDAES, see the [IDAES-PSE Github Page](https://github.com/IDAES/IDAES-PSE) or [IDAES Read-the-docs](https://idaes-pse.readthedocs.io/en/stable/).

## The Auto-thermal Reformer Process

The figure below shows the reformer process as modeled in IDAES.

![Reformer Flowsheet](../images/reformer.png)

This model has 12 outputs of interest, the steam flowrate, the reformer duty, and the properties of the outlet stream, including temperature, pressure, and composition. We are interested modeling how these outputs change as a function of two operating (or input) variables: 
- the fraction of natural gas that bypasses the reformer
- steam to natural gas flow ratio

We have already used IDAES to generate a CSV file that contains the input and output data for 2800 data points for our system.

In this example, we will train a ReLU model from our process data and then demonstrate that we can solve an optimization problem with that surrogate model. In realistic applications, this surrogate model would form part of a design or operations problem with a much larger flowsheet. 

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # suppress CUDA warnings from tensorflow

# import the necessary packages
from omlt import OmltBlock, OffsetScaling
from omlt.io.keras_reader import load_keras_sequential
from omlt.neuralnet import ReluBigMFormulation
import pyomo.environ as pyo
import pandas as pd
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

In [2]:
# read in our csv data
columns = ['Bypass Fraction', 'NG Steam Ratio', 'Steam Flow',
           'Reformer Duty','AR', 'C2H6', 'C3H8', 'C4H10',
           'CH4', 'CO', 'CO2', 'H2', 'H2O', 'N2']
df = pd.read_csv('../data/reformer.csv', usecols=columns)
print(df)

      Bypass Fraction  NG Steam Ratio  Steam Flow  Reformer Duty        AR  \
0                 0.8        0.800000    0.193898    9806.732716  0.002662   
1                 0.8        0.810526    0.196449    9846.047501  0.002660   
2                 0.8        0.821053    0.199000    9885.419259  0.002657   
3                 0.8        0.831579    0.201552    9924.849127  0.002654   
4                 0.8        0.842105    0.204103    9964.338177  0.002651   
...               ...             ...         ...            ...       ...   
2795              0.1        1.157895    1.262887   39771.876388  0.004086   
2796              0.1        1.168421    1.274368   39989.582852  0.004080   
2797              0.1        1.178947    1.285849   40207.531167  0.004073   
2798              0.1        1.189474    1.297330   40425.721366  0.004067   
2799              0.1        1.200000    1.308811   40644.153425  0.004060   

          C2H6      C3H8     C4H10       CH4        CO       CO

In [3]:
# separate the data into inputs and outputs
inputs = ['Bypass Fraction', 'NG Steam Ratio']
outputs = [ 'Steam Flow', 'Reformer Duty','AR', 'C2H6', 'C3H8', 'C4H10',
           'CH4', 'CO', 'CO2', 'H2', 'H2O', 'N2']
dfin = df[inputs]
dfout = df[outputs]

In [4]:
# We scale the data for improved training, however, we want to formulate
# our optimizaton problem on the original variables. Therefore, we keep
# the scaling parameters to use later in our optimization formulation

x_offset, x_factor = dfin.mean().to_dict(), dfin.std().to_dict()
y_offset, y_factor = dfout.mean().to_dict(), dfout.std().to_dict()

dfin = (dfin - dfin.mean()).divide(dfin.std())
dfout = (dfout - dfout.mean()).divide(dfout.std())

# capture the minimum and maximum values of the scaled inputs
# so we don't use the model outside the valid range
scaled_lb = dfin.min()[inputs].values
scaled_ub = dfin.max()[inputs].values

In [6]:
# create our Keras Sequential model
nn = Sequential(name='reformer_relu_4_20')
nn.add(Dense(units=10, input_dim=len(inputs), activation='relu'))
nn.add(Dense(units=10, activation='relu'))
nn.add(Dense(units=10, activation='relu'))
nn.add(Dense(units=10, activation='relu'))
nn.add(Dense(units=len(outputs)))
nn.compile(optimizer=Adam(), loss='mse')

In [7]:
# train our model
x = dfin.values
y = dfout.values

history = nn.fit(x, y, epochs=100)

Epoch 1/100
88/88 [==============================] - 0s 2ms/step - loss: 0.8370
Epoch 2/100
88/88 [==============================] - 0s 2ms/step - loss: 0.4563
Epoch 3/100
88/88 [==============================] - 0s 3ms/step - loss: 0.2696
Epoch 4/100
88/88 [==============================] - 0s 2ms/step - loss: 0.1227
Epoch 5/100
88/88 [==============================] - 0s 2ms/step - loss: 0.0698
Epoch 6/100
88/88 [==============================] - 0s 2ms/step - loss: 0.0440
Epoch 7/100
88/88 [==============================] - 0s 3ms/step - loss: 0.0258
Epoch 8/100
88/88 [==============================] - 0s 3ms/step - loss: 0.0154
Epoch 9/100
88/88 [==============================] - 0s 2ms/step - loss: 0.0103
Epoch 10/100
88/88 [==============================] - 0s 3ms/step - loss: 0.0076
Epoch 11/100
88/88 [==============================] - 0s 2ms/step - loss: 0.0061
Epoch 12/100
88/88 [==============================] - 0s 3ms/step - loss: 0.0051
Epoch 13/100
88/88 [=================

88/88 [==============================] - 0s 2ms/step - loss: 1.9264e-04
Epoch 99/100
88/88 [==============================] - 0s 2ms/step - loss: 1.7761e-04
Epoch 100/100
88/88 [==============================] - 0s 2ms/step - loss: 1.9651e-04


In [9]:
# save the model to disk
# While not technically necessary, this shows how we can load a previously saved model into
# our optimization formulation)
nn.save('reformer_nn_relu')

INFO:tensorflow:Assets written to: reformer_nn_relu/assets


## Optimization Problem
In this small example, we will formulate a simple optimization problem that seeks to maximize the concentration of Hydrogen in the outlet while placing an upper bound on the Nitrogen concentration.

In [10]:
# first, create the Pyomo model
m = pyo.ConcreteModel()

In [11]:
# create the OmltBlock to hold the neural network model
m.reformer = OmltBlock()

In [12]:
# load the Keras model
nn_reformer = keras.models.load_model('reformer_nn_relu', compile=False)

# Note: The neural network is in the scaled space. We want access to the
# variables in the unscaled space. Therefore, we need to tell OMLT about the
# scaling factors
scaler = OffsetScaling(
        offset_inputs={i: x_offset[inputs[i]] for i in range(len(inputs))},
        factor_inputs={i: x_factor[inputs[i]] for i in range(len(inputs))},
        offset_outputs={i: y_offset[outputs[i]] for i in range(len(outputs))},
        factor_outputs={i: y_factor[outputs[i]] for i in range(len(outputs))}
    )

scaled_input_bounds = {i: (scaled_lb[i], scaled_ub[i]) for i in range(len(inputs))}

# create a network definition from the Keras model
net = load_keras_sequential(nn_reformer, scaling_object=scaler, scaled_input_bounds=scaled_input_bounds)

# create the variables and constraints for the neural network in Pyomo
m.reformer.build_formulation(ReluBigMFormulation(net))

In [13]:
# now add the objective and the constraints
h2_idx = outputs.index('H2')
n2_idx = outputs.index('N2')
m.obj = pyo.Objective(expr=m.reformer.outputs[h2_idx], sense=pyo.maximize)
m.con = pyo.Constraint(expr=m.reformer.outputs[n2_idx] <= 0.34)

In [14]:
# now solve the optimization problem (this may take some time)
solver = pyo.SolverFactory('cbc')
status = solver.solve(m, tee=False)

In [15]:
print('Bypass Fraction:', pyo.value(m.reformer.inputs[0]))
print('NG Steam Ratio:', pyo.value(m.reformer.inputs[1]))
print('H2 Concentration:', pyo.value(m.reformer.outputs[h2_idx]))
print('N2 Concentration:', pyo.value(m.reformer.outputs[n2_idx]))

Bypass Fraction: 0.1
NG Steam Ratio: 1.1186717
H2 Concentration: 0.33157189
N2 Concentration: 0.34
